In [1]:
from langchain_community.retrievers import BM25Retriever
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_cohere import CohereRerank
from dotenv import load_dotenv

load_dotenv()

c:\RAG\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
%pip install langchain_cohere

Note: you may need to restart the kernel to use updated packages.


SETUP: Create expanded sample company data (50+ chunks)

In [3]:
chunks = [
    # Tesla - Financial & Production
    "Tesla reported record quarterly revenue of $25.2 billion in Q3 2024.",
    "Tesla's automotive gross margin improved to 19.3% this quarter.",
    "Tesla Cybertruck production ramp begins in 2024 with initial deliveries.",
    "Tesla announced plans to expand Gigafactory production capacity.",
    "Tesla stock price reached new highs following earnings announcement.",
    "Tesla's energy storage business grew 40% year-over-year.",
    "Tesla continues to lead in electric vehicle market share globally.",
    "Tesla Model Y became the best-selling vehicle worldwide.",
    "Tesla reported strong free cash flow generation of $7.5 billion.",
    "Tesla's Full Self-Driving revenue increased significantly.",
    
    # Microsoft - Development & Acquisitions
    "Microsoft acquired GitHub for $7.5 billion in 2018.",
    "Microsoft's cloud revenue Azure grew 29% year-over-year.",
    "Microsoft announced new AI features for Visual Studio Code.",
    "Microsoft Teams integration with GitHub enhances developer workflow.",
    "Microsoft's developer tools division sees strong adoption.",
    "Microsoft acquired Activision Blizzard for $68.7 billion.",
    "Microsoft's productivity suite gained 50 million new users.",
    "Microsoft announced new Surface devices for developers.",
    "Microsoft's AI Copilot features expand to more development tools.",
    "Microsoft's enterprise solutions drive revenue growth.",
    
    # NVIDIA - AI & Hardware
    "NVIDIA's data center revenue reached $47.5 billion annually.",
    "NVIDIA's H100 GPUs see unprecedented demand for AI training.",
    "NVIDIA announced next-generation Blackwell architecture.",
    "NVIDIA's gaming revenue declined due to crypto market changes.",
    "NVIDIA's automotive AI platform partnerships expanded.",
    "NVIDIA's AI chip shortage affects cloud providers.",
    "NVIDIA stock valuation exceeds $2 trillion market cap.",
    "NVIDIA's CUDA platform dominates AI development.",
    "NVIDIA announced new AI inference chips for edge computing.",
    "NVIDIA's partnership with major cloud providers strengthens.",
    
    # Google/Alphabet - AI & Cloud
    "Google's AI investments total over $100 billion in recent years.",
    "Google Cloud revenue grew 35% reaching $8.4 billion quarterly.",
    "Google announced Gemini AI model competing with GPT-4.",
    "Google's search advertising revenue remains strong at $59 billion.",
    "Google's Workspace products integrate advanced AI features.",
    "Google announced quantum computing breakthroughs.",
    "Google's autonomous vehicle division Waymo expands operations.",
    "Google's AI research published breakthrough papers.",
    "Google's cloud AI services see enterprise adoption.",
    "Google faces regulatory scrutiny over AI dominance.",
    
    # Noisy/Less Relevant Chunks
    "The Tesla coil was invented by Nikola Tesla in 1891.",
    "Microsoft Excel spreadsheet formulas can be complex for beginners.",
    "NVIDIA Shield TV streaming device gets software update.",
    "Google Maps navigation improved with real-time traffic data.",
    "Production delays affected multiple manufacturing sectors.",
    "Financial markets showed volatility during earnings season.",
    "Revenue recognition standards changed for software companies.",
    "Hardware components face supply chain constraints globally.",
    "Development tools market grows with remote work trends.",
    "AI research requires significant computational resources.",
    "Quarterly reports show mixed results across tech sector.",
    "Stock market analysts upgrade technology sector ratings.",
    "Cloud computing adoption accelerates in enterprise market.",
    "Data center construction increases globally.",
    "Semiconductor shortage impacts various industries.",
    "Electric vehicle charging infrastructure expands rapidly.",
    "Software development productivity tools gain popularity.",
    "Machine learning frameworks become more accessible.",
    "Enterprise software licensing models evolve.",
    "Technology conferences showcase latest innovations."
]

print(f"Created {len(chunks)} sample chunks for demonstration")

Created 60 sample chunks for demonstration


In [4]:
# Convert to Document objects
documents = [Document(page_content=chunk, metadata={"source": f"chunk_{i}"}) for i, chunk in enumerate(chunks)]

SETUP: Create the three types of retrievers

1. Vector Retriever (Semantic Search/Dense Retrieval)

In [5]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_metadata={"hnsw:space": "cosine"}
)
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 15})

2. BM25 Retriever (Keyword Search/Sparse Retrieval)

In [6]:
# 2. BM25 Retriever
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 15

3. Hybrid Retriever

In [7]:
from langchain_core.runnables import RunnableLambda

print("Setting up Hybrid Retriever...")

def hybrid_search(query: str, k: int = 4):
    vector_docs = vector_retriever.invoke(query)
    bm25_docs = bm25_retriever.invoke(query)

    scores = {}
    doc_store = {}

    # Vector retriever (weight = 0.7)
    for rank, doc in enumerate(vector_docs):
        key = doc.page_content
        scores[key] = scores.get(key, 0) + 0.7 * (1 / (rank + 1))
        doc_store[key] = doc

    # BM25 retriever (weight = 0.3)
    for rank, doc in enumerate(bm25_docs):
        key = doc.page_content
        scores[key] = scores.get(key, 0) + 0.3 * (1 / (rank + 1))
        doc_store[key] = doc

    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)

    return [doc_store[key] for key, _ in ranked[:k]]

hybrid_retriever = RunnableLambda(hybrid_search)

print("Setup complete!\n")


Setting up Hybrid Retriever...
Setup complete!



Step 1: Get results from hybrid search (retrieve more chunks)

In [8]:
query = "Tesla financial performance and production updates"

print("STEP 1: Hybrid Search Results")
print("-"*50)

retrieved_docs = hybrid_retriever.invoke(query)  # Get top 25 for reranking

# Show top 10 from hybrid search
for i, doc in enumerate(retrieved_docs, 1):
    print(f"{i:2d}. {doc.page_content}")

print(f"\n(Retrieved {len(retrieved_docs)} total chunks for reranking)\n")

STEP 1: Hybrid Search Results
--------------------------------------------------
 1. Tesla reported record quarterly revenue of $25.2 billion in Q3 2024.
 2. Tesla announced plans to expand Gigafactory production capacity.
 3. Tesla's automotive gross margin improved to 19.3% this quarter.
 4. Tesla's energy storage business grew 40% year-over-year.

(Retrieved 4 total chunks for reranking)



Step 2: Apply Cohere Reranking

In [9]:
print("STEP 2: After Cohere Reranking (Top 10)")
print("-"*50)

# Initialize Cohere reranker
reranker = CohereRerank(model="rerank-english-v3.0", top_n=10)

# Rerank the retrieved documents
reranked_docs = reranker.compress_documents(retrieved_docs, query)

# Show reranked results
for i, doc in enumerate(reranked_docs, 1):
    print(f"{i:2d}. {doc.page_content}")

print("\n" + "="*80)
print("ANALYSIS:")
print("✅ Hybrid Search: Mixed relevant and irrelevant results")
print("✅ Reranking: Most relevant Tesla financial/production info at top")
print("✅ Notice how reranking moved the most contextually relevant chunks higher")

# Optional: Show the difference more clearly
print("\n" + "="*80)
print("KEY IMPROVEMENTS AFTER RERANKING:")
print("-"*40)


hybrid_top_5 = [doc.page_content for doc in retrieved_docs[:5]]
reranked_top_5= [doc.page_content for doc in reranked_docs[:5]]

print("BEFORE (Hybrid Top 3):")
for i, content in enumerate(hybrid_top_5, 1):
    print(f"  {i}. {content}")

print("\nAFTER (Reranked Top 3):")
for i, content in enumerate(reranked_top_5, 1):
    print(f"  {i}. {content}")

STEP 2: After Cohere Reranking (Top 10)
--------------------------------------------------
 1. Tesla reported record quarterly revenue of $25.2 billion in Q3 2024.
 2. Tesla's automotive gross margin improved to 19.3% this quarter.
 3. Tesla announced plans to expand Gigafactory production capacity.
 4. Tesla's energy storage business grew 40% year-over-year.

ANALYSIS:
✅ Hybrid Search: Mixed relevant and irrelevant results
✅ Reranking: Most relevant Tesla financial/production info at top
✅ Notice how reranking moved the most contextually relevant chunks higher

KEY IMPROVEMENTS AFTER RERANKING:
----------------------------------------
BEFORE (Hybrid Top 3):
  1. Tesla reported record quarterly revenue of $25.2 billion in Q3 2024.
  2. Tesla announced plans to expand Gigafactory production capacity.
  3. Tesla's automotive gross margin improved to 19.3% this quarter.
  4. Tesla's energy storage business grew 40% year-over-year.

AFTER (Reranked Top 3):
  1. Tesla reported record quarte

FINAL RAG: Generate answer with reranked context

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama

print("\n" + "="*80)
print("FINAL: RAG with Reranked Context")
print("-"*40)

# Use top 5 reranked documents for final answer
top_reranked = reranked_docs[:5]

combined_input = f"""Based on the following documents, please answer this question: {query}

Documents:
{chr(10).join([f"- {doc.page_content}" for doc in top_reranked])}

Please provide a clear, helpful answer using only the information from these documents."""

model = ChatOllama(model="llama3.2")
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content=combined_input),
]

result = model.invoke(messages)
print("Generated Response:")
print(result.content)


FINAL: RAG with Reranked Context
----------------------------------------
Generated Response:
Based on the provided documents, here are the key financial performance and production updates for Tesla:

**Financial Performance:**

1. Record quarterly revenue of $25.2 billion in Q3 2024.
2. Automotive gross margin improved to 19.3% this quarter.

**Production Updates:**

1. Expansion plans announced for Gigafactory production capacity, although no specific details or timelines are provided.
2. Energy storage business saw a 40% year-over-year growth.
